In [2]:
import librosa
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

In [ ]:
import os
import numpy as np
import librosa
data_path = 'D:/release_in_the_wild/'
def load_and_preprocess_data(data_path):
    audio_features = []
    labels = []
    file_count = 0

    # Check if the dataset path exists
    if not os.path.exists(dataset_path):
        print(f"Dataset path {dataset_path} does not exist.")
        return np.array([]), np.array([])

    # Walk through the dataset directory
    for root, dirs, files in os.walk(dataset_path):
        print(f"Processing directory: {root}")
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(root, file)
                try:
                    audio, sr = librosa.load(file_path, sr=16000)
                    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
                    mfcc_mean = np.mean(mfcc.T, axis=0)
                    audio_features.append(mfcc_mean)
                    # labels.append(0 if 'real' in file_path else 1)
                    # file_count += 1

                    # if file_count % 5000 == 0:
                    #     print(f'Processed {file_count} files')

                except Exception as e:
                    print(f'Error processing file {file_path}: {e}')

    return np.array(audio_features), np.array(labels)

dataset_path = 'D:/release_in_the_wild/'
X, y = load_and_preprocess_data(dataset_path)

print(f'Extracted features shape: {X.shape}')
print(f'Labels shape: {y.shape}')

In [ ]:
from sklearn.preprocessing import StandardScaler
#Data normalisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import LeakyReLU, GaussianNoise
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def build_model(input_shape):
    model = Sequential()

    # Input layer with Gaussian noise for regularization
    model.add(GaussianNoise(0.1, input_shape=(input_shape,)))

    # First hidden layer
    model.add(Dense(128, kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.3))

    # Second hidden layer
    model.add(Dense(64, kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.4))

    # Third hidden layer
    model.add(Dense(32, kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))

    # Optimizer with learning rate scheduling
    optimizer = Adam(learning_rate=0.001, decay=1e-6)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model